# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'02-11-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'02-11-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-02-12 05:23:29,33.93911,67.709953,55420,2419,48279,4722.0,Afghanistan,142.364145,4.364850
1,NaN,NaN,NaN,Albania,2021-02-12 05:23:29,41.15330,20.168300,89776,1517,54362,33897.0,Albania,3119.605254,1.689761
2,NaN,NaN,NaN,Algeria,2021-02-12 05:23:29,28.03390,1.659600,110049,2930,75436,31683.0,Algeria,250.960963,2.662450
3,NaN,NaN,NaN,Andorra,2021-02-12 05:23:29,42.50630,1.521800,10391,106,9781,504.0,Andorra,13448.521323,1.020114
4,NaN,NaN,NaN,Angola,2021-02-12 05:23:29,-11.20270,17.873900,20261,487,18710,1064.0,Angola,61.646792,2.403633


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,55121,55174,55231,55265,55330,55335,55359,55384,55402,55420
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,79934,80941,81993,83082,84212,85336,86289,87528,88671,89776
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,107841,108116,108381,108629,108629,109088,109313,109559,109782,110049


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2405,2406,2407,2407,2408,2410,2413,2414,2418,2419
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1398,1404,1418,1433,1446,1460,1472,1488,1503,1517
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2898,2900,2904,2909,2909,2914,2918,2924,2926,2930


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,47798,47982,47995,48018,48027,48041,48042,48109,48133,48279
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,48377,48958,49543,50219,50875,51444,52128,52933,53616,54362
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,73732,73940,74136,74330,74330,74692,74871,75057,75243,75436


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,5723,5753,5811,5824,5856,5869,5881,5910,5930,5970
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,18344,18418,18494,18568,18668,18723,18763,18824,18888,18960
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1966,1981,1989,1994,2002,2008,2008,2019,2024,2030


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,70,71,74,76,77,77,77,77,79,81
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,228,228,233,234,227,227,227,228,239,240
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,40,42,43,44,45,45,45,45,46,46


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
651,1001.0,Autauga,Alabama,US,2021-02-12 05:23:29,32.539527,-86.644082,5970,81,0,5889.0,"Autauga, Alabama, US",10685.711217,1.356784
688,1075.0,Lamar,Alabama,US,2021-02-12 05:23:29,33.779950,-88.096680,1275,32,0,1243.0,"Lamar, Alabama, US",9235.784136,2.509804
689,1077.0,Lauderdale,Alabama,US,2021-02-12 05:23:29,34.901719,-87.656247,8432,196,0,8236.0,"Lauderdale, Alabama, US",9093.163951,2.324478


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,27390465,475291,0
India,10880603,155447,10589230
Brazil,9713909,236201,8637050


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,27390465,475291,0,26915174,2021-02-12 05:23:29,37.936303,-91.379001
India,10880603,155447,10589230,135926,2021-02-12 05:23:29,23.088275,81.806127
Brazil,9713909,236201,8637050,840658,2021-02-12 05:23:29,-12.669522,-48.480493
United Kingdom,4010376,115748,10598,3884030,2021-02-12 05:23:29,30.744028,-35.367255
Russia,3983031,77415,3499230,406386,2021-02-12 05:23:29,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3459706,45869,0
Texas,2543236,40535,0
Florida,1806805,28382,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3459706,45869,0,3413837,2021-02-12 05:23:29,37.843962,-120.728594
Texas,2543236,40535,0,2502701,2021-02-12 05:23:29,31.660643,-98.653069
Florida,1806805,28382,0,1778423,2021-02-12 05:23:29,28.940755,-82.700744
New York,1514070,45453,0,1468617,2021-02-12 05:23:29,42.544151,-75.474183
Illinois,1155833,21985,0,1133848,2021-02-12 05:23:29,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1158619,18658,0
Arizona,Maricopa,494345,8306,0
Illinois,Cook,463581,9100,0
Florida,Miami-Dade,389519,5081,0
Texas,Harris,331837,4664,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1158619,18658,0,1139961,2021-02-12 05:23:29,34.308284,-118.228241,6037.0
Arizona,Maricopa,494345,8306,0,486039,2021-02-12 05:23:29,33.348359,-112.491815,4013.0
Illinois,Cook,463581,9100,0,454481,2021-02-12 05:23:29,41.841448,-87.816588,17031.0
Florida,Miami-Dade,389519,5081,0,384438,2021-02-12 05:23:29,25.611236,-80.551706,12086.0
Texas,Harris,331837,4664,0,327173,2021-02-12 05:23:29,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-09,55384,87528,109559,10312,20163,316,1993295,168300,28871,426093,...,3983756,46153,79204,1,131096,2064,163975,2131,64610,34781
2021-02-10,55402,88671,109782,10352,20210,350,2001034,168496,28879,427562,...,3996833,46778,79237,1,131476,2091,164736,2133,65573,34864
2021-02-11,55420,89776,110049,10391,20261,381,2008345,168676,28887,429139,...,4010376,47254,79303,1,131828,2140,165535,2134,66598,34864


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-09,2414,1488,2924,106,478,8,49566,3126,909,8071,...,114066,506,621,0,1247,35,1897,615,881,1353
2021-02-10,2418,1503,2926,106,487,9,49674,3130,909,8114,...,115068,516,621,0,1253,35,1909,616,901,1364
2021-02-11,2419,1517,2930,106,487,9,49874,3135,909,8138,...,115748,524,622,0,1260,35,1910,616,914,1364


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-09,48109,52933,75057,9674,18623,189,1792321,160169,26105,404676,...,10427,40272,77609,1,122991,1472,153682,1428,58319,29289
2021-02-10,48133,53616,75243,9732,18687,190,1798120,160405,26108,405921,...,10495,40769,77648,1,123411,1480,154260,1428,58875,29471
2021-02-11,48279,54362,75436,9781,18710,197,1806260,160670,26112,407155,...,10598,41426,77726,1,123811,1528,154727,1428,59989,29471


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,5723,5753,5811,5824,5856,5869,5881,5910,5930,5970
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1251,1253,1260,1264,1267,1269,1272,1275,1275,1275
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,8216,8255,8298,8332,8350,8366,8380,8399,8411,8432
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2665,2682,2699,2708,2717,2723,2729,2734,2746,2755
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,13958,14048,14149,14197,14277,14337,14361,14391,14443,14491


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-09,474666,55893,0,787268,308848,3442672,407210,264608,1429,49,...,747462,2518333,356040,13105,2485,534116,324025,125522,602256,52830
2021-02-10,476067,56097,0,789245,309940,3451176,407961,265496,81461,49,...,750409,2531060,357339,13122,2485,537319,324706,125951,603255,52874
2021-02-11,477570,56251,0,791106,311043,3459706,409683,266499,81875,49,...,752033,2543236,358581,13249,2505,541018,326159,126420,604683,52979


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-02-09        5910   18824    2019  2362   5929    1132   1872   12477   
2021-02-10        5930   18888    2024  2368   5937    1131   1882   12498   
2021-02-11        5970   18960    2030  2377   5955    1136   1886   12539   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-02-09         3283     1735  ...    2552    562     2954      667   
2021-02-10         3291     1738  ...    2551    561     2957      668   
2021-02-11         3305     1738  ...    2555    563     2961      669   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-09           3561  3266  1994          0      870    618  
2021-02-10           3571  3249  1995          0      871    617  
2021-02-11           3586  3256  1992          0      872    617  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-09,8579,280,0,14286,5148,45009,5746,7298,1221,0,...,10631,39799,1748,186,24,6898,4558,2150,6663,647
2021-02-10,8888,280,0,14462,5174,45496,5770,7326,1245,0,...,10731,40148,1765,187,24,6932,4603,2175,6705,647
2021-02-11,9021,280,0,14662,5199,45869,5781,7354,1262,0,...,10812,40535,1774,188,25,6958,4633,2187,6723,647


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-02-09          77     228      45   53    113      29     62     247   
2021-02-10          79     239      46   54    116      31     64     257   
2021-02-11          81     240      46   54    116      32     64     257   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-02-09           84       37  ...      25     11       23        7   
2021-02-10           89       37  ...      25     11       23        7   
2021-02-11           92       37  ...      25     11       23        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-09             34     9    12          0       26      5  
2021-02-10             34     8    12          0       26      5  
2021-02-11             34     8    12          0       26      5  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-09,0.000452,0.014359,0.002250,0.003601,0.002536,0.000000,0.003925,0.000731,0.000381,0.002817,...,0.003133,0.011019,0.000531,0.0,0.003829,0.006829,0.004669,0.000000,0.016312,0.003549
2021-02-10,0.000325,0.013059,0.002035,0.003879,0.002331,0.107595,0.003883,0.001165,0.000277,0.003448,...,0.003283,0.013542,0.000417,0.0,0.002899,0.013081,0.004641,0.000939,0.014905,0.002386
2021-02-11,0.000325,0.012462,0.002432,0.003767,0.002524,0.088571,0.003654,0.001068,0.000277,0.003688,...,0.003388,0.010176,0.000833,0.0,0.002677,0.023434,0.004850,0.000469,0.015631,0.000000


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-09,0.000414,0.010870,0.002056,0.0,0.006316,0.142857,0.003401,0.000961,0.0,0.004856,...,0.009309,0.018109,0.00000,NaN,0.005645,0.0,0.004767,0.000000,0.013809,0.010456
2021-02-10,0.001657,0.010081,0.000684,0.0,0.018828,0.125000,0.002179,0.001280,0.0,0.005328,...,0.008784,0.019763,0.00000,NaN,0.004812,0.0,0.006326,0.001626,0.022701,0.008130
2021-02-11,0.000414,0.009315,0.001367,0.0,0.000000,0.000000,0.004026,0.001597,0.0,0.002958,...,0.005910,0.015504,0.00161,NaN,0.005587,0.0,0.000524,0.000000,0.014428,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-09,0.001395,0.015443,0.002484,0.006660,0.001452,0.000000,0.003439,0.001833,0.000307,0.003407,...,0.010075,0.014996,0.000580,0.0,0.003754,0.000000,0.004333,0.0,0.035328,0.016097
2021-02-10,0.000499,0.012903,0.002478,0.005995,0.003437,0.005291,0.003235,0.001473,0.000115,0.003077,...,0.006522,0.012341,0.000503,0.0,0.003415,0.005435,0.003761,0.0,0.009534,0.006214
2021-02-11,0.003033,0.013914,0.002565,0.005035,0.001231,0.036842,0.004527,0.001652,0.000153,0.003040,...,0.009814,0.016115,0.001005,0.0,0.003241,0.032432,0.003027,0.0,0.018921,0.000000


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-09,0.002784,0.002241,NaN,0.005596,0.004799,0.003084,0.002299,0.003295,0.000700,0.0,...,0.002194,0.005492,0.002585,0.004522,0.007705,0.006200,0.002509,0.003325,0.001438,0.000871
2021-02-10,0.002952,0.003650,NaN,0.002511,0.003536,0.002470,0.001844,0.003356,56.005598,0.0,...,0.003943,0.005054,0.003648,0.001297,0.000000,0.005997,0.002102,0.003418,0.001659,0.000833
2021-02-11,0.003157,0.002745,NaN,0.002358,0.003559,0.002472,0.004221,0.003778,0.005082,0.0,...,0.002164,0.004811,0.003476,0.009678,0.008048,0.006884,0.004475,0.003724,0.002367,0.001986


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-09      0.004931  0.003251  0.005478  0.006820  0.001520  0.000000   
2021-02-10      0.003384  0.003400  0.002476  0.002540  0.001349 -0.000883   
2021-02-11      0.006745  0.003812  0.002964  0.003801  0.003032  0.004421   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-09      0.002141  0.004104  0.005205  0.002311  ...  0.000784   
2021-02-10      0.005342  0.001683  0.002437  0.001729  ... -0.000392   
2021-02-11      0.002125  0.003281  0.004254  0.000000  ...  0.001568   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-02-09     -0.001776  0.001017  0.000000   0.000562  0.002763  0.003018   
2021-02-10     -0.001779  0.001016  0.001499   0.002808 -0.005205  0.000502   
2021-02-11      0.003565  0.001353  0.001497   0.004201  0.002155 -0.001504   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-02-09            NaN  0.000000  0.000000  
2021-02-10            NaN  0.001149 -0.001618  
2021-02-11            NaN  0.001148  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-09,0.006570,0.003584,NaN,0.016435,0.008226,0.011575,0.002268,0.002197,0.010762,NaN,...,0.006152,0.008029,0.005754,0.016393,0.000000,0.011437,0.024040,0.008916,0.007409,0.036859
2021-02-10,0.036018,0.000000,NaN,0.012320,0.005051,0.010820,0.004177,0.003837,0.019656,NaN,...,0.009406,0.008769,0.009725,0.005376,0.000000,0.004929,0.009873,0.011628,0.006303,0.000000
2021-02-11,0.014964,0.000000,NaN,0.013829,0.004832,0.008199,0.001906,0.003822,0.013655,NaN,...,0.007548,0.009639,0.005099,0.005348,0.041667,0.003751,0.006517,0.005517,0.002685,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-09      0.000000  0.004405  0.000000  0.000000  0.008929  0.000000   
2021-02-10      0.025974  0.048246  0.022222  0.018868  0.026549  0.068966   
2021-02-11      0.025316  0.004184  0.000000  0.000000  0.000000  0.032258   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2021-02-09      0.000000  0.004065  0.037037  0.027778  ...  0.086957    0.0   
2021-02-10      0.032258  0.040486  0.059524  0.000000  ...  0.000000    0.0   
2021-02-11      0.000000  0.000000  0.033708  0.000000  ...  0.000000    0.0   

Province_State                                                           \
Admin2          Sheridan Sublette Sweetwater     Teton Uinta Unassigned   
2021-02-09      0.045455      0.0   0.030303  0.285714   0.0        NaN   
2021-02-10      0.000000      0.0   0.000000 -0.111111   0.0        NaN   
2021-02-11      0.000000      0.0   0.000000  0.000000   0.0        NaN   

Province_State                  
Admin2         Washakie Weston  
2021-02-09          0.0    0.0  
2021-02-10          0.0    0.0  
2021-02-11          0.0    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-09,0.000469,0.013297,0.002317,0.003434,0.001976,0.024042,0.003331,0.000747,0.000262,0.002831,...,0.003571,0.010256,0.000646,0.0,0.003720,0.011650,0.004101,0.000589,0.016896,0.003391
2021-02-10,0.000397,0.013178,0.002176,0.003657,0.002153,0.065819,0.003607,0.000956,0.000269,0.003139,...,0.003427,0.011899,0.000531,0.0,0.003309,0.012366,0.004371,0.000764,0.015900,0.002888
2021-02-11,0.000361,0.012820,0.002304,0.003712,0.002338,0.077195,0.003630,0.001012,0.000273,0.003414,...,0.003408,0.011037,0.000682,0.0,0.002993,0.017900,0.004611,0.000616,0.015766,0.001444


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-09,0.000664,0.009834,0.001677,0.002218,0.003511,0.071510,0.003452,0.001289,3.130144e-17,0.003823,...,0.006867,0.016226,6.569048e-09,NaN,0.005363,2.721193e-50,0.004316,9.331733e-09,0.015542,0.010071
2021-02-10,0.001160,0.009957,0.001180,0.001109,0.011170,0.098255,0.002816,0.001284,1.565072e-17,0.004575,...,0.007826,0.017995,3.284524e-09,NaN,0.005087,1.360597e-50,0.005321,8.130128e-04,0.019122,0.009101
2021-02-11,0.000787,0.009636,0.001274,0.000555,0.005585,0.049127,0.003421,0.001441,7.825361e-18,0.003767,...,0.006868,0.016749,8.051546e-04,NaN,0.005337,6.802983e-51,0.002922,4.065064e-04,0.016775,0.004550


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-09,0.000810,0.014046,0.002616,0.006614,0.002274,0.004619,0.003830,0.001494,0.000260,0.003309,...,0.008494,0.015486,0.000562,0.0,0.003806,0.000527,0.003666,0.000047,0.030446,0.012653
2021-02-10,0.000654,0.013474,0.002547,0.006305,0.002855,0.004955,0.003532,0.001484,0.000187,0.003193,...,0.007508,0.013914,0.000532,0.0,0.003610,0.002981,0.003714,0.000023,0.019990,0.009433
2021-02-11,0.001844,0.013694,0.002556,0.005670,0.002043,0.020898,0.004030,0.001568,0.000170,0.003116,...,0.008661,0.015014,0.000768,0.0,0.003426,0.017707,0.003371,0.000012,0.019456,0.004717


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-09,0.002705,0.003672,NaN,0.004352,0.003885,0.003375,0.002553,0.006175,0.001803,1.556408e-100,...,0.002403,0.004758,0.002416,0.007824,0.004942,0.005659,0.004101,0.003554,0.001404,0.001412
2021-02-10,0.002828,0.003661,NaN,0.003432,0.003710,0.002923,0.002199,0.004765,28.003701,7.782042e-101,...,0.003173,0.004906,0.003032,0.004561,0.002471,0.005828,0.003101,0.003486,0.001531,0.001122
2021-02-11,0.002993,0.003203,NaN,0.002895,0.003635,0.002697,0.003210,0.004272,14.004391,3.891021e-101,...,0.002668,0.004858,0.003254,0.007119,0.005260,0.006356,0.003788,0.003605,0.001949,0.001554


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-09      0.003915  0.003138  0.003605  0.004638  0.001933  0.002538   
2021-02-10      0.003650  0.003269  0.003041  0.003589  0.001641  0.000827   
2021-02-11      0.005197  0.003541  0.003003  0.003695  0.002337  0.002624   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-09      0.002183  0.003458  0.003365  0.001666  ...  0.000848   
2021-02-10      0.003762  0.002571  0.002901  0.001697  ...  0.000228   
2021-02-11      0.002944  0.002926  0.003578  0.000849  ...  0.000898   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-02-09      0.000528  0.001950  0.000750   0.002103  0.003764  0.002146   
2021-02-10     -0.000626  0.001483  0.001125   0.002456 -0.000720  0.001324   
2021-02-11      0.001470  0.001418  0.001311   0.003328  0.000717 -0.000090   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-02-09           -1.0  0.000039  0.001369  
2021-02-10           -1.0  0.000594 -0.000125  
2021-02-11           -1.0  0.000871 -0.000062  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-09,0.005086,0.002047,NaN,0.009811,0.006558,0.009448,0.001814,0.003699,0.008404,NaN,...,0.006900,0.006619,0.004110,0.009579,3.163460e-13,0.008416,0.012613,0.006442,0.004437,0.018614
2021-02-10,0.020552,0.001023,NaN,0.011065,0.005804,0.010134,0.002995,0.003768,0.014030,NaN,...,0.008153,0.007694,0.006918,0.007478,1.581730e-13,0.006672,0.011243,0.009035,0.005370,0.009307
2021-02-11,0.017758,0.000512,NaN,0.012447,0.005318,0.009166,0.002451,0.003795,0.013842,NaN,...,0.007851,0.008667,0.006008,0.006413,2.083333e-02,0.005212,0.008880,0.007276,0.004027,0.004654


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-09      0.002503  0.000888  0.002912  0.001223  0.006572  0.000001   
2021-02-10      0.014238  0.024567  0.012567  0.010045  0.016560  0.034483   
2021-02-11      0.019777  0.014375  0.006283  0.005023  0.008280  0.033371   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-09      0.000521  0.003189  0.019021  0.014010  ...  0.044068   
2021-02-10      0.016389  0.021837  0.039272  0.007005  ...  0.022034   
2021-02-11      0.008195  0.010919  0.036490  0.003502  ...  0.011017   

Province_State                                                             \
Admin2                Platte  Sheridan      Sublette Sweetwater     Teton   
2021-02-09      3.078251e-06  0.023346  2.516269e-12   0.015276  0.143514   
2021-02-10      1.539125e-06  0.011673  1.258134e-12   0.007638  0.016202   
2021-02-11      7.695626e-07  0.005836  6.290672e-13   0.003819  0.008101   

Province_State                                              
Admin2                 Uinta Unassigned  Washakie   Weston  
2021-02-09      4.082018e-07  -0.501961  0.000156  0.00332  
2021-02-10      2.041009e-07  -0.501961  0.000078  0.00166  
2021-02-11      1.020505e-07  -0.501961  0.000039  0.00083  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210211,AK,54134,NaN,NaN,NaN,totalTestsViral,1577356,33.0,1227.0,...,54134,0,3,9bde9e7688ef2df020f5194474be7cce451af917,0,0,0,0,0,NaN
1,20210211,AL,477570,102824.0,1836625.0,NaN,totalTestsPeopleViral,2211371,1281.0,43906.0,...,2314195,133,221,243e7ec9a9859fad20d054cd855496c6af9e9a5f,0,0,0,0,0,NaN
2,20210211,AR,311043,64454.0,2316686.0,NaN,totalTestsViral,2563275,712.0,14255.0,...,2627729,25,44,43a63659cb0009325dbe62edbddf8d30e632f740,0,0,0,0,0,NaN
3,20210211,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,f9af3096de9a7a18426442a07acd27ac851f35e6,0,0,0,0,0,NaN
4,20210211,AZ,791106,52978.0,2855265.0,NaN,totalTestsViral,7105696,2507.0,55272.0,...,3646371,200,184,edf01773882f81a0ff47a4fd11097f5b24b3b5d3,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210211,AK,54134.0,NaN,NaN,NaN,totalTestsViral,1577356.0,33.0,1227.0,...,54134,0,3,9bde9e7688ef2df020f5194474be7cce451af917,0,0,0,0,0,NaN
1,20210211,AL,477570.0,102824.0,1836625.0,NaN,totalTestsPeopleViral,2211371.0,1281.0,43906.0,...,2314195,133,221,243e7ec9a9859fad20d054cd855496c6af9e9a5f,0,0,0,0,0,NaN
2,20210211,AR,311043.0,64454.0,2316686.0,NaN,totalTestsViral,2563275.0,712.0,14255.0,...,2627729,25,44,43a63659cb0009325dbe62edbddf8d30e632f740,0,0,0,0,0,NaN
3,20210211,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,f9af3096de9a7a18426442a07acd27ac851f35e6,0,0,0,0,0,NaN
4,20210211,AZ,791106.0,52978.0,2855265.0,NaN,totalTestsViral,7105696.0,2507.0,55272.0,...,3646371,200,184,edf01773882f81a0ff47a4fd11097f5b24b3b5d3,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-02-11,AK,54134,NaN,NaN,NaN,totalTestsViral,1577356,33.0,1227.0,NaN,...,54134,0,3,9bde9e7688ef2df020f5194474be7cce451af917,0,0,0,0,0,NaN
2021-02-11,AL,477570,102824.0,1836625.0,NaN,totalTestsPeopleViral,2211371,1281.0,43906.0,NaN,...,2314195,133,221,243e7ec9a9859fad20d054cd855496c6af9e9a5f,0,0,0,0,0,NaN
2021-02-11,AR,311043,64454.0,2316686.0,NaN,totalTestsViral,2563275,712.0,14255.0,256.0,...,2627729,25,44,43a63659cb0009325dbe62edbddf8d30e632f740,0,0,0,0,0,NaN
2021-02-11,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,f9af3096de9a7a18426442a07acd27ac851f35e6,0,0,0,0,0,NaN
2021-02-11,AZ,791106,52978.0,2855265.0,NaN,totalTestsViral,7105696,2507.0,55272.0,719.0,...,3646371,200,184,edf01773882f81a0ff47a4fd11097f5b24b3b5d3,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-02-11,AK,54134.0,NaN,NaN,NaN,totalTestsViral,1577356.0,33.0,1227.0,NaN,...,54134,0,3,9bde9e7688ef2df020f5194474be7cce451af917,0,0,0,0,0,NaN
2021-02-11,AL,477570.0,102824.0,1836625.0,NaN,totalTestsPeopleViral,2211371.0,1281.0,43906.0,NaN,...,2314195,133,221,243e7ec9a9859fad20d054cd855496c6af9e9a5f,0,0,0,0,0,NaN
2021-02-11,AR,311043.0,64454.0,2316686.0,NaN,totalTestsViral,2563275.0,712.0,14255.0,256.0,...,2627729,25,44,43a63659cb0009325dbe62edbddf8d30e632f740,0,0,0,0,0,NaN
2021-02-11,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,f9af3096de9a7a18426442a07acd27ac851f35e6,0,0,0,0,0,NaN
2021-02-11,AZ,791106.0,52978.0,2855265.0,NaN,totalTestsViral,7105696.0,2507.0,55272.0,719.0,...,3646371,200,184,edf01773882f81a0ff47a4fd11097f5b24b3b5d3,0,0,0,0,0,NaN
